In [1]:
# Installing dependencies
!pip3 install opencv-python
!pip3 install numpy

Cloning repository of all black box adversarial attack methods.

In [2]:
!git clone https://github.com/SCLBD/BlackboxBench.git

fatal: destination path 'BlackboxBench' already exists and is not an empty directory.


See if we can just plug our dataset into the methods and run it.

Load Drive and test example on it

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%%writefile /content/my_detector_module.py
import torch
from torch import nn
from torchvision import models, transforms
import numpy as np
import cv2

# replicate your model definition
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x, None)
        return fmap, self.dp(self.linear1(x_lstm[:, -1, :]))

# checkpoint path (change this to your Drive model path)
_CHECKPOINT_PATH = "/content/drive/My Drive/Models/model_87_acc_20_frames_final_data.pt"

# instantiate model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
_model = Model(2).to(device)
_model.load_state_dict(torch.load(_CHECKPOINT_PATH, map_location=device))
_model.eval()

# same preprocessing as in your notebook
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
_sm = nn.Softmax(dim=1)

# prediction wrapper for attack
def predict_batch(np_batch):
    """
    np_batch: numpy array (N,H,W,3) uint8
    returns: list of predicted labels (0=fake,1=real)
    """
    results = []
    with torch.no_grad():
        for frame in np_batch:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            tensor = _transform(frame).unsqueeze(0).unsqueeze(0).to(device)
            fmap, logits = _model(tensor)
            probs = _sm(logits)
            preds = torch.argmax(probs, dim=1)
            results.append(preds.item())
    return np.array(results)


Overwriting /content/my_detector_module.py


All in one

In [5]:
# == Plug-and-play cell: run your repo's BoundaryAttack on first 70 frames using your trained detector ==
REPO_PATH = "/content/BlackboxBench/query"
MODULE_WITH_PREDICT = "/content/my_detector_module.py"  # change path if stored in Drive

# ---------------------------- do not change below ----------------------------
%pylab inline
import os, sys, importlib.util, traceback
from typing import List, Tuple
import numpy as np
import cv2
import torch
import torch.nn.functional as F

try:
    import torchvision
    import torchvision.transforms.functional as TF
except Exception:
    torchvision = None
    TF = None

def ensure_package(root):
    for d in ["", "attacks", "attacks/decision"]:
        p = os.path.join(root, d)
        if os.path.isdir(p):
            init = os.path.join(p, "__init__.py")
            if not os.path.exists(init):
                open(init, "a").close()
ensure_package(REPO_PATH)

if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

boundary_attack_path = os.path.join(REPO_PATH, "attacks/decision/boundary_attack.py")
BoundaryAttack = None
if os.path.exists(boundary_attack_path):
    spec = importlib.util.spec_from_file_location("boundary_attack", boundary_attack_path)
    module = importlib.util.module_from_spec(spec)
    try:
        sys.modules["boundary_attack"] = module
        spec.loader.exec_module(module)
        BoundaryAttack = getattr(module, "BoundaryAttack", None)
        print("Imported BoundaryAttack:", BoundaryAttack)
    except Exception as e:
        print("Failed to import boundary_attack module:", e)
        traceback.print_exc()
else:
    print("boundary_attack.py not found at:", boundary_attack_path)

def safe_get_fps(cap):
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps is None or fps <= 0 or np.isnan(fps):
        return 25.0
    return float(fps)

def Process_Video(INPUT_VIDEO, OUTPUT_VIDEO, NUM_CHANGE):
    if not os.path.exists(INPUT_VIDEO):
        raise FileNotFoundError("Upload your input video to /content/ and set INPUT_VIDEO path.")

    cap = cv2.VideoCapture(INPUT_VIDEO)
    if not cap.isOpened():
        raise RuntimeError("Failed to open input video.")
    fps = safe_get_fps(cap)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frames_bgr = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames_bgr.append(frame)
    cap.release()
    print(f"Loaded {len(frames_bgr)} frames (W x H = {w} x {h}), fps={fps}")

    attack = None
    if BoundaryAttack is not None:
        try:
            attack = BoundaryAttack(
                epsilon=1e-1,
                p='2',
                max_queries=5,
                lb=0.0,
                ub=255.0,
                batch_size=2,
                steps=5,
                spherical_step=0.1,
                source_step=0.01,
                source_step_convergance=1e-4,
                step_adaptation=1.5,
                update_stats_every_k=10,
            )
            print("Instantiated BoundaryAttack.")
        except Exception as e:
            print("Failed to instantiate BoundaryAttack:", e)
            traceback.print_exc()
            attack = None
    else:
        print("BoundaryAttack not available; will fallback per-frame.")

    # --------------------------------------------------------------------
    # use your trained model module instead of ResNet18
    # --------------------------------------------------------------------
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_input_size = (224, 224)

    def model_predict_labels_from_candidate_tensor(candidate_tensor):
        # dynamically import your detector
        import importlib.util
        spec = importlib.util.spec_from_file_location("user_detector_module", MODULE_WITH_PREDICT)
        user_mod = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(user_mod)

        if isinstance(candidate_tensor, torch.Tensor):
            np_batch = candidate_tensor.detach().cpu().numpy()
        else:
            np_batch = np.asarray(candidate_tensor)
        if np_batch.ndim == 3:
            np_batch = np_batch[None, ...]
        if np_batch.dtype != np.uint8:
            if np_batch.max() <= 1.0:
                np_batch = (np_batch * 255.0).clip(0,255).astype(np.uint8)
            else:
                np_batch = np.clip(np_batch, 0,255).astype(np.uint8)

        if hasattr(user_mod, "predict_batch"):
            out = user_mod.predict_batch(np_batch)
        elif hasattr(user_mod, "predict"):
            out = user_mod.predict(np_batch)
        else:
            raise RuntimeError("Module must provide predict_batch(np_batch) or predict(np_batch).")

        out = np.asarray(out)
        if out.ndim == 1 and np.issubdtype(out.dtype, np.floating):
            labels = (out > 0.5).astype(np.int64)
        elif out.ndim == 2 and out.shape[1] == 2:
            labels = np.argmax(out, axis=1).astype(np.int64)
        else:
            labels = out.astype(np.int64).reshape(-1)

        return torch.from_numpy(labels).long()

    def batch_to_model_input(x_batch, target_size=model_input_size):
        if isinstance(x_batch, np.ndarray):
            x = torch.from_numpy(x_batch.astype(np.float32))
        else:
            x = x_batch.float()
        x = x.to(device)
        x = x.permute(0,3,1,2)
        x = x / 255.0
        x = F.interpolate(x, size=target_size, mode="bilinear", align_corners=False)
        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
        std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
        x = (x - mean) / std
        return x

    if attack is not None:
        def is_adversarial_fn(candidates, labels):
            if isinstance(candidates, torch.Tensor):
                cand = candidates.detach().cpu().numpy()
            else:
                cand = np.asarray(candidates)
            preds = model_predict_labels_from_candidate_tensor(cand)
            if isinstance(labels, torch.Tensor):
                labs = labels.detach().cpu().long()
            else:
                labs = torch.tensor(labels, dtype=torch.long)
            return (preds != labs).to(torch.bool)

        def distance_fn(a, b):
            if not isinstance(a, torch.Tensor):
                a = torch.from_numpy(np.asarray(a).astype(np.float32))
            if not isinstance(b, torch.Tensor):
                b = torch.from_numpy(np.asarray(b).astype(np.float32))
            a = a.view(a.shape[0], -1)
            b = b.view(b.shape[0], -1)
            return torch.norm(a - b, dim=1)

        attack.is_adversarial = is_adversarial_fn
        attack.distance = distance_fn
        print("Patched attack.is_adversarial and attack.distance to use your detector.")
    else:
        print("Attack object not available.")

    # --------------------------------------------------------------------
    # process frames
    # --------------------------------------------------------------------
    out_frames = []
    changed = 0

    for i, frame_bgr in enumerate(frames_bgr):
        if i < NUM_CHANGE and attack is not None:
            try:
                frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
                xs = torch.from_numpy(frame_rgb.astype(np.float32)).unsqueeze(0)
                orig_label = model_predict_labels_from_candidate_tensor(xs)
                ys = orig_label
                adv, q = attack._perturb(xs, ys)
                if isinstance(adv, torch.Tensor):
                    adv_np = adv.detach().cpu().numpy()
                else:
                    adv_np = np.asarray(adv)
                if adv_np.ndim == 4 and adv_np.shape[0] == 1:
                    adv_np = adv_np[0]
                adv_np = np.clip(adv_np, 0, 255).astype(np.uint8)
                out_bgr = cv2.cvtColor(adv_np, cv2.COLOR_RGB2BGR)
                out_frames.append(out_bgr)
                changed += 1
                print(f"Frame {i}: attack applied (q={q}).")
            except Exception as e:
                print(f"Frame {i}: attack failed -> fallback invert. Error: {e}")
                traceback.print_exc()
                out_frames.append(255 - frame_bgr)
        else:
            break

    print(f"Processed {len(out_frames)} frames; changed {changed} frames.")

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))
    if not writer.isOpened():
        raise RuntimeError("Failed to open VideoWriter for output.")
    for f in out_frames:
        writer.write(f)
    writer.release()
    print("Saved output video to:", OUTPUT_VIDEO)


Populating the interactive namespace from numpy and matplotlib
Imported BoundaryAttack: <class 'boundary_attack.BoundaryAttack'>


In [6]:
# # == Plug-and-play cell: run your repo's BoundaryAttack on first 70 frames ==
# REPO_PATH = "/content/BlackboxBench/query"


# # ---------------------------- do not change below ----------------------------
# import os, sys, importlib.util, traceback
# from typing import List, Tuple
# import numpy as np
# import cv2
# import torch
# import torch.nn.functional as F

# # Try to import torchvision for a pretrained decision oracle
# try:
#     import torchvision
#     import torchvision.transforms.functional as TF
# except Exception:
#     torchvision = None
#     TF = None

# # ensure repo packages have __init__.py so imports work
# def ensure_package(root):
#     for d in ["", "attacks", "attacks/decision"]:
#         p = os.path.join(root, d)
#         if os.path.isdir(p):
#             init = os.path.join(p, "__init__.py")
#             if not os.path.exists(init):
#                 open(init, "a").close()
# ensure_package(REPO_PATH)

# # make repo importable
# if REPO_PATH not in sys.path:
#     sys.path.insert(0, REPO_PATH)

# # attempt to import boundary_attack.py
# boundary_attack_path = os.path.join(REPO_PATH, "attacks/decision/boundary_attack.py")
# BoundaryAttack = None
# if os.path.exists(boundary_attack_path):
#     spec = importlib.util.spec_from_file_location("boundary_attack", boundary_attack_path)
#     module = importlib.util.module_from_spec(spec)
#     try:
#         sys.modules["boundary_attack"] = module
#         spec.loader.exec_module(module)
#         BoundaryAttack = getattr(module, "BoundaryAttack", None)
#         print("Imported BoundaryAttack:", BoundaryAttack)
#     except Exception as e:
#         print("Failed to import boundary_attack module:", e)
#         traceback.print_exc()
# else:
#     print("boundary_attack.py not found at:", boundary_attack_path)

# # load video frames
# def safe_get_fps(cap):
#     fps = cap.get(cv2.CAP_PROP_FPS)
#     if fps is None or fps <= 0 or np.isnan(fps):
#         return 25.0
#     return float(fps)

# def Process_Video(INPUT_VIDEO, OUTPUT_VIDEO, NUM_CHANGE):
#   if not os.path.exists(INPUT_VIDEO):
#       raise FileNotFoundError("Upload your input video to /content/ and set INPUT_VIDEO path.")

#   cap = cv2.VideoCapture(INPUT_VIDEO)
#   if not cap.isOpened():
#       raise RuntimeError("Failed to open input video.")
#   fps = safe_get_fps(cap)
#   w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#   h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#   frames_bgr = []
#   while True:
#       ret, frame = cap.read()
#       if not ret:
#           break
#       frames_bgr.append(frame)
#   cap.release()
#   print(f"Loaded {len(frames_bgr)} frames (W x H = {w} x {h}), fps={fps}")

#   # prepare attack instance (use string '2' for p)
#   attack = None
#   if BoundaryAttack is not None:
#       try:
#           attack = BoundaryAttack(
#               epsilon=1e-2,
#               p='2',                    # <- IMPORTANT: string '2'
#               max_queries=10,
#               lb=0.0,
#               ub=255.0,
#               batch_size=3,
#               steps=5,
#               spherical_step=0.1,
#               source_step=0.01,
#               source_step_convergance=1e-4,
#               step_adaptation=1.5,
#               update_stats_every_k=10,
#           )
#           print("Instantiated BoundaryAttack.")
#       except Exception as e:
#           print("Failed to instantiate BoundaryAttack:", e)
#           traceback.print_exc()
#           attack = None
#   else:
#       print("BoundaryAttack not available; will fallback per-frame.")

#   # load pretrained model as decision oracle (ResNet18 ImageNet)
#   device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#   model = None
#   model_input_size = (224, 224)
#   if torchvision is not None:
#       try:
#           model = torchvision.models.resnet18(pretrained=True).to(device).eval()
#           print("Loaded torchvision resnet18 as decision oracle.")
#       except Exception as e:
#           print("Failed to load pretrained model:", e)
#           model = None
#   else:
#       print("torchvision not available; attack will fallback.")

#   # helpers to convert candidates to model inputs & predict labels
#   def batch_to_model_input(x_batch, target_size=model_input_size):
#       # x_batch: torch tensor shape (N,H,W,3) in [0,255] or numpy array
#       if isinstance(x_batch, np.ndarray):
#           x = torch.from_numpy(x_batch.astype(np.float32))
#       else:
#           x = x_batch.float()
#       x = x.to(device)
#       x = x.permute(0,3,1,2)  # (N,3,H,W)
#       x = x / 255.0
#       x = F.interpolate(x, size=target_size, mode="bilinear", align_corners=False)
#       mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
#       std  = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
#       x = (x - mean) / std
#       return x

#   def model_predict_labels_from_candidate_tensor(candidate_tensor):
#       # accepts torch tensor (N,H,W,3) or numpy, returns torch.LongTensor (N,)
#       if model is None:
#           raise RuntimeError("No model available for predictions.")
#       was_numpy = isinstance(candidate_tensor, np.ndarray)
#       if was_numpy:
#           cand = torch.from_numpy(candidate_tensor.astype(np.float32)).to(device)
#       else:
#           cand = candidate_tensor.to(device).float()
#       x = batch_to_model_input(cand, target_size=model_input_size)
#       with torch.no_grad():
#           logits = model(x)
#           preds = torch.argmax(logits, dim=1)
#       return preds.cpu()

#   # monkey-patch attack.is_adversarial and attack.distance if attack exists
#   if attack is not None and model is not None:
#       def is_adversarial_fn(candidates, labels):
#           # candidates: torch Tensor (N,H,W,3) or numpy -> returns torch.BoolTensor (N,)
#           if isinstance(candidates, torch.Tensor):
#               cand = candidates.detach().cpu()
#           else:
#               cand = np.asarray(candidates)
#           preds = model_predict_labels_from_candidate_tensor(cand)
#           if isinstance(labels, torch.Tensor):
#               labs = labels.detach().cpu().long()
#           else:
#               labs = torch.tensor(labels, dtype=torch.long)
#           return (preds != labs).to(torch.bool)

#       def distance_fn(a, b):
#           # L2 distance per-sample; a,b are torch tensors or numpy
#           if not isinstance(a, torch.Tensor):
#               a = torch.from_numpy(np.asarray(a).astype(np.float32))
#           if not isinstance(b, torch.Tensor):
#               b = torch.from_numpy(np.asarray(b).astype(np.float32))
#           a = a.view(a.shape[0], -1)
#           b = b.view(b.shape[0], -1)
#           return torch.norm(a - b, dim=1)

#       attack.is_adversarial = is_adversarial_fn
#       attack.distance = distance_fn
#       print("Patched attack.is_adversarial and attack.distance.")
#   else:
#       print("Attack or model not available -> falling back per-frame where needed.")

#   # process frames: attempt attack on first NUM_CHANGE frames
#   out_frames = []
#   changed = 0

#   for i, frame_bgr in enumerate(frames_bgr):
#       if i < NUM_CHANGE and attack is not None and model is not None:
#           try:
#               frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
#               xs = torch.from_numpy(frame_rgb.astype(np.float32)).unsqueeze(0)  # (1,H,W,3)
#               # get the model's predicted label for the original frame (so attack tries to change it)
#               orig_label = model_predict_labels_from_candidate_tensor(xs)  # (1,)
#               ys = orig_label  # use predicted label as the "true" label
#               adv, q = attack._perturb(xs, ys)  # call repo attack
#               # convert adv to numpy HWC uint8
#               if isinstance(adv, torch.Tensor):
#                   adv_np = adv.detach().cpu().numpy()
#               else:
#                   adv_np = np.asarray(adv)
#               if adv_np.ndim == 4 and adv_np.shape[0] == 1:
#                   adv_np = adv_np[0]
#               adv_np = np.clip(adv_np, 0, 255).astype(np.uint8)
#               out_bgr = cv2.cvtColor(adv_np, cv2.COLOR_RGB2BGR)
#               out_frames.append(out_bgr)
#               changed += 1
#               print(f"Frame {i}: attack applied (q={q}).")
#           except Exception as e:
#               print(f"Frame {i}: attack failed -> fallback invert. Error: {e}")
#               traceback.print_exc()
#               out_frames.append(255 - frame_bgr)
#       else:
#           break

#   print(f"Processed {len(out_frames)} frames; changed {changed} frames.")

#   # write output video
#   fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#   writer = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))
#   if not writer.isOpened():
#       raise RuntimeError("Failed to open VideoWriter for output.")
#   for f in out_frames:
#       writer.write(f)
#   writer.release()
#   print("Saved output video to:", OUTPUT_VIDEO)


In [7]:
import os
import glob
from tqdm import tqdm

In [ ]:
# Set these paths before running:
REPO_PATH = "/content/BlackboxBench/query"
src_dir = "/content/drive/MyDrive/faceforensics++/manipulated_sequences/DeepFakeDetection/c40/videos/"
output_dir = "/content/drive/MyDrive/faceforensics++/Adversarial_attacked_sequences/QueryBasedAttacks/BoundaryAttack/DeepfakeDetectionSet/Epsilon0.1/"
NUM_CHANGE = 70

for video_path in tqdm(glob.glob(os.path.join(src_dir, "*.mp4"))):
  fname = os.path.basename(video_path)
  if os.path.exists(output_dir + fname):
    continue
  Process_Video(src_dir + fname, output_dir + fname, NUM_CHANGE)

  0%|          | 0/3068 [00:00<?, ?it/s]

Loaded 1620 frames (W x H = 1920 x 1080), fps=24.0
Instantiated BoundaryAttack.
Patched attack.is_adversarial and attack.distance to use your detector.


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Frame 0: attack applied (q=5).
Frame 1: attack applied (q=5).
Frame 2: attack applied (q=5).
Frame 3: attack applied (q=5).
Mean Squared Error: tensor([197027.5625])
Calls: 2
Frame 4: attack applied (q=5).
Frame 5: attack applied (q=5).
Frame 6: attack applied (q=5).
Mean Squared Error: tensor([210277.5156])
Calls: 4
Frame 7: attack applied (q=5).
Frame 8: attack applied (q=5).
